# Data Preprocessing

### feature of voice data

- 음성 파일을 전처리 하는 방법에 대해 알아보자. 음성 파일을 바라보는 관점은 크게 2가지로 나눌 수 있다. wav단위와 segment단위이다. wav단위는 음성 파일 전체를 하나로 바라보는것, segment는 wav단위 음성 파일을 여러 구간으로 쪼개어 사용하는 것이다. 일반적으로 모델 학습은 segment, 추론은 wav단위로 진행한다.

### data processing

- 따라서 모델에 훈련시키기 위해, wav파일을 segment 단위로 잘라서, 해당 단위의 feature을 계산하여 모델이 쓸 수 있는 tensor로 반환해야 한다.

- (1) raw waveform : 통째로, 일반적으로 pretraining 경우 사용
- (2) mel spectrogram : 주파수를 사람 귀 기준으로 변환. 음성의 발음, 톤, 주파수 패턴을 잘 표현
- (3) MFCC : 고전 방식으로 베이스 라인 용으로 사용.
- (4) Spectral features : 공간적 특징들로, centroid, bandwidth, rolloff, flux로, 소리 밝기, 주파수, 고주파 비율, 프레임간 변화량의 특징이다. 일반적으로 mel(2)번과 concat하여 사용한다. deepvoice에 최적화된 특징으로 알려져 있다.
- (5) prosody : 리듬 억양 특징으로, pitch, energy/rms, speaking rate로 보이스피싱 감정, 자연성 판단에 중요하다.

---

아래 예시 코드를 보고 어떤식으로 진행하는지 확인하자.

# 1.import

- (1) import os : 파일을 쉽게 읽기 위한 모듈
- (2) import librosa : wav/flac 음성 파일을 로딩하고, 특징들을 추출하는 함수 제공하는 모듈
- (3) import torch : 최종 feature를 tensor로 변환하기 위한 모듈
- (4) from torch.utlis.data import Dataset : 앞서 정의한 Dataset을 사용하기 위한 모듈이다. 이때 우리가 사용하는 Dataset을 통해 class를 만든다면 위 Dataset을 반드시 상속해야한다. 또한 __len__ 과 __getitem__ 메서드를 반드시 포함해야한다.

In [ ]:
import os
import librosa
import torch
import numpy as np
from torch.utils.data import Dataset

# 2.Class

## 2.1 Class

- voice를 입력받아 segment 단위로 처리하는 class 코드를 확인하자.
위에서 말한것 처럼, Dataset을 상속하는 클래스이다.

## 2.2 def \__init__

### argument 
- (1) mode : train 과 test를 모두 처리하기 위함이다.
- (2) cfg : cfg는 yaml에서 로드한 설정 딕셔너리이다.

### setting value
- (1) self.cfg = cfg
- (2) self.mode = mode
- (3) self.segment_frames = cfg["data"]["segment_frames"]
    - config.yaml의 dict형태 접근 코드
- (4) self.sr = cfg["data"]["sample_rate"]
    - sr은 샘플레이트로 1초에 소리를 몇번 찍느냐이다. 일반적으로 16khz, 16000 Hz를 사용한다.
- (5) self.hop = 512  
    - 예컨데, 10초짜리 음성을 2 segment로 분할했다 하자. 그렇다면 0~2, 2~4, 4~6, 6~8. 8~10으로 5구간이 overlap없이 분할된다. 만약 hop을 1초로 설정한다면, 0~2, 1~3, 2~4 .. 로 9개의 segment가 생성된다. 즉 hop은 다음 frame간의 간격을 의미한다.

In [ ]:
class SegmentVoiceDataset(Dataset):
    def __init__(self, cfg, mode="train"):
        """
        mode: 'train' or 'test'
        """
        self.cfg = cfg
        self.mode = mode
        self.segment_frames = cfg["data"]["segment_frames"]
        self.sr = cfg["data"]["sample_rate"]
        self.hop = 512  # librosa default hop

        self.items = []
        if mode == "train":
            self._prepare_train()
        elif mode == "test":
            self._prepare_test()
        else:
            raise ValueError("mode must be 'train' or 'test'")

        print(f"[Dataset] {mode} segments: {len(self.items)}")

## def _prepare_train

- 위 함수에서는 학습 데이터를 구성한다.


In [ ]:
def _prepare_train(self):
        root = self.cfg["data"]["train_path"]
        #root 폴더 config.yaml에서 읽어오기.
        wav_id = 0

        wav_paths = []
        for r, _, files in os.walk(root):
            #os.walk를 통해 flac 파일만 읽어 올 수 있다.
            for f in files:
                if f.endswith(".flac"):
                    wav_paths.append(os.path.join(r, f))
                    #r은 root, f는 실제 파일 1개이다.

        wav_paths = wav_paths[:5]
        #디버깅을 위해 실제 파일중 5개만 사용한다.

        for path in wav_paths:
            wav, _ = librosa.load(path, sr=self.sr)
            #만약 불러온 wav_paths의 path = "001.flac"의 파일이
            # 5초짜리라면 flac파일을 읽고 mono 변환, sample rate를 맞춘후,
            # 1차원 wav를 nd array로 반환한다.
            # 예상 5초 * 16000 샘플레이트 = 80000 array 반환
            #이때 반환된 80000개의 값은 5초짜리 음성의 진폭을 숫자로 기록한것.
            #즉 1초에 16000번 5초동안 소리를 찍어서 기록했다.
            n_frames = len(wav) // self.hop
            # 반환된 wav의 길이를 hop으로 나누어 하나의 wav파일에 대한 frame 저장.
            # 80000길의 array를 156개의 frame으로 저장한것이다.
            n_segments = n_frames // self.segment_frames
            # 이때 각 프레임을 또 segment_frames으로 나눌 수 있다.
            # 만약 segment_frames = 64로 설정했다고 가정하자.
            # 그럼 하나의 frame에 2개의 segment가 나오고, 그 segment하나에
            # ...
            for i in range(n_segments):
                self.items.append({
                    "path": path,
                    "label": 0,
                    "start": i * self.segment_frames,
                    "wav_id": wav_id
                })

            wav_id += 1   